<a href="https://colab.research.google.com/github/skylar1218/ML-Project---Predicting-Disease-Spread/blob/master/src/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sources
Data Preparation: https://www.dataquest.io/blog/pandas-python-tutorial/

# Import Data

In [0]:
# import package
import numpy as np
import pandas as pd

In [0]:
# import csv files
url = "https://raw.githubusercontent.com/skylar1218/ML-Project---Predicting-Disease-Spread/master/data/dengue_features_train.csv"
dengue_features_train = pd.read_csv(url)

In [0]:
url = "https://raw.githubusercontent.com/skylar1218/ML-Project---Predicting-Disease-Spread/master/data/dengue_labels_train.csv"
dengue_labels_train = pd.read_csv(url)

In [0]:
url = 'https://raw.githubusercontent.com/skylar1218/ML-Project---Predicting-Disease-Spread/master/data/dengue_features_test.csv'
dengue_features_test = pd.read_csv(url)

# Data Description

In [6]:
dengue_features_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [38]:
dengue_features_train.shape

(1456, 24)

In [0]:
dengue_labels_train.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [41]:
dengue_labels_train.shape

(1456, 4)

In [0]:
dengue_features_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [42]:
dengue_features_test.shape

(416, 24)

In [8]:
dengue_features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
city                                     1456 non-null object
year                                     1456 non-null int64
weekofyear                               1456 non-null int64
week_start_date                          1456 non-null object
ndvi_ne                                  1262 non-null float64
ndvi_nw                                  1404 non-null float64
ndvi_se                                  1434 non-null float64
ndvi_sw                                  1434 non-null float64
precipitation_amt_mm                     1443 non-null float64
reanalysis_air_temp_k                    1446 non-null float64
reanalysis_avg_temp_k                    1446 non-null float64
reanalysis_dew_point_temp_k              1446 non-null float64
reanalysis_max_air_temp_k                1446 non-null float64
reanalysis_min_air_temp_k                1446 non-null float64
reanalysis_precip

In [9]:
dengue_features_train.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
count,1456.000000,1456.000000,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000
mean,2001.031593,26.503434,0.142294,0.130553,0.203783,0.202305,45.760388,298.701852,299.225578,295.246356,303.427109,295.719156,40.151819,82.161959,45.760388,16.746427,4.903754,27.185783,8.059328,32.452437,22.102150,39.326360
std,5.408314,15.019437,0.140531,0.119999,0.073860,0.083903,43.715537,1.362420,1.261715,1.527810,3.234601,2.565364,43.434399,7.153897,43.715537,1.542494,3.546445,1.292347,2.128568,1.959318,1.574066,47.455314
min,1990.000000,1.000000,-0.406250,-0.456100,-0.015533,-0.063457,0.000000,294.635714,294.892857,289.642857,297.800000,286.900000,0.000000,57.787143,0.000000,11.715714,1.357143,21.400000,4.528571,26.700000,14.700000,0.000000
25%,1997.000000,13.750000,0.044950,0.049217,0.155087,0.144209,9.800000,297.658929,298.257143,294.118929,301.000000,293.900000,13.055000,77.177143,9.800000,15.557143,2.328571,26.300000,6.514286,31.100000,21.100000,8.700000
50%,2002.000000,26.500000,0.128817,0.121429,0.196050,0.189450,38.340000,298.646429,299.289286,295.640714,302.400000,296.200000,27.245000,80.301429,38.340000,17.087143,2.857143,27.414286,7.300000,32.800000,22.200000,23.850000
75%,2005.000000,39.250000,0.248483,0.216600,0.248846,0.246982,70.235000,299.833571,300.207143,296.460000,305.500000,297.900000,52.200000,86.357857,70.235000,17.978214,7.625000,28.157143,9.566667,33.900000,23.300000,53.900000
max,2010.000000,53.000000,0.508357,0.454429,0.538314,0.546017,390.600000,302.200000,302.928571,298.450000,314.000000,299.900000,570.500000,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,25.600000,543.300000


In [10]:
# check missing for each column
dengue_features_train.isnull().sum(axis=0)

city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_mi

# Delete Observations with Majority Features Missing

https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/

In [0]:
# keep only the rows with at least 5 non-na values
# X_train1 = dengue_features_train.dropna(thresh=5)

In [0]:
# get names of indexes for which observations with more than 15 variables' values missing
indexNames = dengue_features_train[dengue_features_train.isnull().sum(axis=1)>15].index

In [46]:
indexNames

Int64Index([87, 139, 399, 451, 711, 763, 1118, 1170, 1378, 1430], dtype='int64')

In [0]:
# delete these row indexes from dengue_features_train
X_train1 = dengue_features_train.drop(indexNames)
y_train1 = dengue_labels_train.drop(indexNames)

In [48]:
print(X_train1.shape)
print(y_train1.shape)

(1446, 24)
(1446, 4)


# Replace Feature Missing Value

https://medium.com/@george.drakos62/handling-missing-values-in-machine-learning-part-1-dda69d4f88ca

https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779

*   Replace with overall Median
```
dataframe.Column_Name.fillna(dataframe.Column_Name.median(),inplace=True)
```
*   Replace with values close to the mean value but within one standard deviation
```
Column_Name_avg = dataframe['Column_Name'].mean()
Column_Name_std = dataframe['Column_Name'].std()
Column_Name_null_count = dataframe['Column_Name'].isnull().sum()
Column_Name_null_random_list = np.random.randint(Column_Name_avg - Column_Name_std, Column_Name_avg + Column_Name_std, size=Column_Name_null_count)
dataframe['Column_Name'][np.isnan(dataframe['Column_Name'])] = Column_Name_null_random_list
dataframe['Column_Name'] = dataframe['Column_Name'].astype(int)
```
*   Back-fill or forward-fill to propagate next or previous values respectively
```
# for back fill
dataframe.fillna(method='bfill',inplace=True)
# for forward-fill
dataframe.fillna(method='ffill',inplace=True)
```
*   Replace with some constant value outside fixed value range: -999,-1 etc
```
dataframe.Column_Name.fillna(-99,inplace=True)
```
*   Replace with median by cluster (imputation using k-NN)
 * Computationally expensive. KNN works by storing the whole training dataset in memory.
 * K-NN is quite sensitive to outliers in the data (unlike SVM).
```
import sys
from impyute.imputation.cs import fast_knn
# Increase the recursion limit of the OS
sys.setrecursionlimit(100000)
# start the KNN training
imputed_training=fast_knn(train.values, k=30)
```
* Imputation Using Multivariate Imputation by Chained Equation (MICE)
 * filling the missing data multiple times
 * Multiple Imputations (MIs) are much better than a single imputation as it measures the uncertainty of the missing values in a better way
  * can handle different variables of different data types (ie., continuous or binary) as well as complexities such as bounds or survey skip patterns
```
from impyute.imputation.cs import mice
# start the MICE training
imputed_training=mice(train.values)
```
* Stochastic regression imputation
 * predict the missing values by regressing it from other related variables in the same dataset plus some random residual value

* Multivariate feature imputation
 * https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer
* Nearest neighbors imputation
 * https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer





In [0]:
# replace missing value with KNN
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

In [0]:
label = X_train1.iloc[:, 0:4]

In [0]:
data = X_train1.iloc[:, 4:]

In [0]:
data_missing_filled = imputer.fit_transform(data)

In [72]:
data_missing_filled

array([[1.226000e-01, 1.037250e-01, 1.984833e-01, ..., 2.940000e+01,
        2.000000e+01, 1.600000e+01],
       [1.699000e-01, 1.421750e-01, 1.623571e-01, ..., 3.170000e+01,
        2.220000e+01, 8.600000e+00],
       [3.225000e-02, 1.729667e-01, 1.572000e-01, ..., 3.220000e+01,
        2.280000e+01, 4.140000e+01],
       ...,
       [2.470571e-01, 1.460571e-01, 2.503571e-01, ..., 3.220000e+01,
        1.920000e+01, 7.400000e+00],
       [3.339143e-01, 2.457714e-01, 2.788857e-01, ..., 3.120000e+01,
        2.100000e+01, 1.600000e+01],
       [2.981857e-01, 2.329714e-01, 2.742143e-01, ..., 3.370000e+01,
        2.220000e+01, 2.040000e+01]])